In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Filter the dataset for birds (label 2) and non-birds (labels 0-1, 3-9)
bird_indices = (y_train == 2).flatten()  # Label 2 corresponds to birds
non_bird_indices = (y_train != 2).flatten()

x_train_birds = x_train[bird_indices]
y_train_birds = y_train[bird_indices]

x_train_non_birds = x_train[non_bird_indices][:5000]  # Taking a subset for balance
y_train_non_birds = y_train[non_bird_indices][:5000]

# Combine bird and non-bird data
x_train_combined = np.concatenate((x_train_birds, x_train_non_birds), axis=0)
y_train_combined = np.concatenate((np.ones(y_train_birds.shape), np.zeros(y_train_non_birds.shape)), axis=0)

# Shuffle the combined dataset
shuffled_indices = np.random.permutation(len(x_train_combined))
x_train_combined = x_train_combined[shuffled_indices]
y_train_combined = y_train_combined[shuffled_indices]

# Normalize the images
x_train_combined = x_train_combined.astype('float32') / 255.0

# Build a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train_combined, y_train_combined, epochs=10, batch_size=64, validation_split=0.2)

# Save the model
model.save('bird_detection_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 98ms/step - accuracy: 0.5474 - loss: 0.6821 - val_accuracy: 0.7050 - val_loss: 0.5781
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 89ms/step - accuracy: 0.6979 - loss: 0.5762 - val_accuracy: 0.7305 - val_loss: 0.5329
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 96ms/step - accuracy: 0.7238 - loss: 0.5361 - val_accuracy: 0.7315 - val_loss: 0.5122
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 94ms/step - accuracy: 0.7544 - loss: 0.4934 - val_accuracy: 0.7760 - val_loss: 0.4801
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 87ms/step - accuracy: 0.7873 - loss: 0.4517 - val_accuracy: 0.7800 - val_loss: 0.4676
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 94ms/step - accuracy: 0.7999 - loss: 0.4221 - val_accuracy: 0.7870 - val_loss: 0.4592
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 95ms/step - accuracy: 0.8053 - loss: 0.4199 - val_accuracy: 0.7635 - val_loss: 0.5083
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 94ms/step - accuracy: 0.8323 - loss: 0.3731 - 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = load_model('bird_detection_model.h5')

def prepare_image(img_path):
    # Load the image from the path
    img = image.load_img(img_path, target_size=(32, 32))  # Resize the image to 32x32 pixels

    # Convert the image to a numpy array
    img_array = image.img_to_array(img)

    # Normalize the image array
    img_array = img_array.astype('float32') / 255.0

    # Add an extra dimension to match the input shape of the model (batch size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)

    return img_array

def predict_bird(img_path):
    # Prepare the image
    img_array = prepare_image(img_path)

    # Make the prediction
    prediction = model.predict(img_array)

    # Interpret the prediction result
    if prediction[0] > 0.5:
        print("Bird detected in the image.")
    else:
        print("No bird detected in the image.")

# Example usage
img_path = 'car.jpg'  # Replace with the path to your image
predict_bird(img_path)
img_path = 'bird.jpg'
predict_bird(img_path)
img_path = 'bird3.jpg'
predict_bird(img_path)
img_path = 'bird2.jpg'
predict_bird(img_path)
img_path = 'bird1.jpg'
predict_bird(img_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
No bird detected in the image.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Bird detected in the image.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
No bird detected in the image.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
No bird detected in the image.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Bird detected in the image.
